<a href="https://colab.research.google.com/github/8dci/T5/blob/main/Artificial_Neural_Networks_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Networks for Binary Classification Exercise

In this notebook we will build an ANN to predict whether customers will churn (indicated by the column `Churn`).


## Dataset
- **Source**: [Kaggle](https://www.kaggle.com/datasets/ahmedgaitani/customer-churn-prediction-dataset/data).
- **Variables of Interest**:
  - `Churn`
- As for the data card you should be able by now to figure the data out by your self :)

## Preprocessing Steps

1. **Dropping irrelevant columns**
2. **Encoding categorical variables**
3. **Data Splitting**

## Model Architecture

- **Input Layer**: Please specify here (the trainee should this this)
- **Hidden Layers**: Please specify here (the trainee should fill this)
- **Output Layer**: Please specify here (the trainee should fill this)

## Training

- **Optimizer**: Please specify here (the trainee should fill this)
- **Loss Function**: Please specify here (the trainee should fill this)
- **Metrics**: Please specify here (the trainee should fill this)
- **Epochs**: Please specify here (the trainee should fill this)
- **Batch Size**: Please specify here (the trainee should fill this)
- **Validation Split**: Please specify here (the trainee should fill this)

## Evaluation
- Please specify here (the trainee should fill this)

If you need to change the steps or add more please do so and explain why!

# Import libraries

customerID: معرف العميل.

gender: جنس العميل.

SeniorCitizen: هل العميل من كبار السن (0 = لا، 1 = نعم).

Partner: هل لدى العميل شريك.

Dependents: هل لدى العميل معالين.

tenure: مدة الخدمة (بالأشهر).

PhoneService: هل يشترك العميل في خدمة الهاتف.

InternetService: نوع خدمة الإنترنت التي يشترك فيها العميل.

Contract: نوع العقد الذي يمتلكه العميل.

MonthlyCharges: الرسوم الشهرية.

TotalCharges: إجمالي الرسوم.

Churn: هل العميل قد غادر الشركة أم لا (Yes/No).

In [2]:
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install matplotlib seaborn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Load Data and EDA

In [4]:
df = pd.read_csv('/content/Churn_Modelling_Exercise.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0001-XXXX,Male,0,Yes,Yes,5,No,Yes,No,No,...,No,No,No,No internet service,Month-to-month,Yes,Credit card,27.43,137.15,Yes
1,0002-XXXX,Female,0,No,No,42,Yes,No phone service,DSL,Yes,...,No,No,No internet service,No internet service,Two year,No,Electronic check,38.28,1607.76,Yes
2,0003-XXXX,Male,0,No,No,61,Yes,No phone service,No,No,...,No,Yes,Yes,No,One year,No,Bank transfer,106.44,6492.84,Yes
3,0004-XXXX,Male,1,No,Yes,22,Yes,No,No,No internet service,...,Yes,No,No internet service,Yes,Month-to-month,No,Electronic check,92.49,2034.78,No
4,0005-XXXX,Male,1,Yes,Yes,21,No,No,DSL,No,...,No internet service,No,No,No internet service,One year,No,Bank transfer,19.63,412.23,Yes


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        300 non-null    object 
 1   gender            300 non-null    object 
 2   SeniorCitizen     300 non-null    int64  
 3   Partner           300 non-null    object 
 4   Dependents        300 non-null    object 
 5   tenure            300 non-null    int64  
 6   PhoneService      300 non-null    object 
 7   MultipleLines     300 non-null    object 
 8   InternetService   300 non-null    object 
 9   OnlineSecurity    300 non-null    object 
 10  OnlineBackup      300 non-null    object 
 11  DeviceProtection  300 non-null    object 
 12  TechSupport       300 non-null    object 
 13  StreamingTV       300 non-null    object 
 14  StreamingMovies   300 non-null    object 
 15  Contract          300 non-null    object 
 16  PaperlessBilling  300 non-null    object 
 1

In [6]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,300.000000,300.000000,300.000000,300.000000
mean,0.543333,35.783333,67.226800,2477.975767
std,0.498951,21.113400,28.638073,1918.540111
min,0.000000,1.000000,18.450000,30.030000
25%,0.000000,17.000000,42.337500,910.302500
50%,1.000000,36.000000,68.635000,2072.400000
75%,1.000000,53.250000,92.430000,3618.405000
max,1.000000,72.000000,118.640000,8318.880000


In [9]:
df.duplicated().sum()

0

In [10]:
df.isna().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [13]:
df.drop(['customerID','SeniorCitizen'],axis=1 ,inplace=True)

## Preprocessing

### Drop unnecessary columns

### One-hot-encoder and Label-encoder

In [14]:
encoding_columns = ['Partner',]
ohe_encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = ohe_encoder.fit_transform(df[encoding_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=ohe_encoder.get_feature_names_out(encoding_columns))
df_encoded = pd.concat([df, one_hot_df], axis=1)


df_encoded = df_encoded.drop(encoding_columns, axis=1)

df = df_encoded

In [16]:
label_encoder = LabelEncoder()

# Encode labels in column 'Gender'.
df['gender']= label_encoder.fit_transform(df['gender'])

In [30]:
encoding_columns1 = ['Dependents',]
ohe_encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = ohe_encoder.fit_transform(df[encoding_columns1])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=ohe_encoder.get_feature_names_out(encoding_columns1))
df_encoded = pd.concat([df, one_hot_df], axis=1)


df_encoded = df_encoded.drop(encoding_columns1, axis=1)

df = df_encoded

In [33]:
label_encoder1 = LabelEncoder()
df['PhoneService']= label_encoder1.fit_transform(df['PhoneService'])

In [37]:
from sklearn.preprocessing import LabelEncoder

object_cols = df.select_dtypes(include=['object']).columns

label_encoder = LabelEncoder()
for col in object_cols:
    df[col] = label_encoder.fit_transform(df[col])

print(df.head())


   gender  tenure  PhoneService  MultipleLines  InternetService  \
0       1       5             0              2                2   
1       0      42             1              1                0   
2       1      61             1              1                2   
3       1      22             1              0                2   
4       1      21             0              0                0   

   OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport  StreamingTV  \
0               0             2                 0            0            0   
1               2             2                 0            0            1   
2               0             2                 0            2            2   
3               1             1                 2            0            1   
4               0             0                 1            0            0   

   ...  Contract  PaperlessBilling  PaymentMethod  MonthlyCharges  \
0  ...         0                 1              1    

## Selecting Features && Train Test Split

### Selecting Features

In [38]:
X = df.drop('Churn', axis=1)
y = df['Churn']

### Train Test Split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Modeling

### Creating the sequential model

In [40]:
model=Sequential()

### Adding the input layer

In [41]:
model.add(Dense(units=6, input_dim=X_train.shape[1], kernel_initializer='uniform',activation='relu'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Adding the hidden layers

In [42]:
model.add(Dense(units=6,activation='relu'))
model.add(Dense(units=6,activation='relu'))

### Adding the output layer

In [43]:
model.add(Dense(units=1,activation='sigmoid'))

### Compiling the model

In [44]:
model.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

### Display model summary

In [45]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 6)                   │             126 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 6)                   │              42 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 6)                   │              42 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │               7 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 217 (868.00 B)

 Trainable params: 217 (868.00 B)

 Non-trainable params: 0 (0.00 B)

### Fitting the model

In [46]:
model.fit(X_train,y_train,batch_size= 10 ,epochs=100,validation_split=0.2)

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5398 - loss: 1.2125 - val_accuracy: 0.4048 - val_loss: 0.7261
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6349 - loss: 0.6735 - val_accuracy: 0.5238 - val_loss: 0.6981
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6136 - loss: 0.6776 - val_accuracy: 0.4286 - val_loss: 0.7192
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6581 - loss: 0.6717 - val_accuracy: 0.5000 - val_loss: 0.7016
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5972 - loss: 0.6894 - val_accuracy: 0.4286 - val_loss: 0.7241
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6006 - loss: 0.6655 - val_accuracy: 0.4762 - val_loss: 0.7029
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6192 - loss: 0.6877 - val_accuracy: 0.4048 - val_loss: 0.7174
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5857 - loss: 0.6868 - val_accuracy: 0.4286 - 

## Evaluating the model

In [47]:
model.evaluate(X_test,y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5440 - loss: 0.7205 


[0.728107213973999, 0.5333333611488342]